Import Required modules

In [ ]:
# import requried modules

import numpy as np
import pandas as pd

from xgboost import XGBClassifier

from matplotlib import pyplot

from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn import metrics
from sklearn.compose import ColumnTransformer

from sklearn.linear_model import LogisticRegression
from sklearn.multioutput import MultiOutputClassifier

from sklearn.pipeline import Pipeline

from sklearn.model_selection import train_test_split

from sklearn.metrics import roc_curve, roc_auc_score, classification_report, confusion_matrix, f1_score

from numpy import mean
from sklearn.datasets import make_classification
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.tree import DecisionTreeClassifier
from imblearn.pipeline import Pipeline
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from collections import Counter

from numpy import loadtxt
from numpy import sort
from xgboost import XGBClassifier
from xgboost import plot_importance
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.feature_selection import SelectFromModel

from collections import OrderedDict
from itertools import product
import random
import os
from importlib import reload
from sklearn.ensemble import RandomForestClassifier

Load Data

In [ ]:
RANDOM_SEED = 1    # Set a random seed for reproducibility!

taxidata = pd.read_csv('../input/newdata/train_new.csv', nrows = 100_000, index_col='tripid')
testdata = pd.read_csv('../input/newdata/test_new.csv', nrows = 100_000, index_col='tripid')

taxidata.isnull().sum(axis = 0)
taxidata=taxidata.dropna(how='any',axis=0)
# taxidata=taxidata.dropna(how='any',axis=0, subset=['fare'])
# taxidata=taxidata.fillna(value={'fare': 0, 'additional_fare':0,	'duration':0,	'meter_waiting':0, 'meter_waiting_fare':0,	'meter_waiting_till_pickup':0})
"""Convert correct incorrect strings into Boolean values"""
mat_t = {'correct': 1, 'incorrect': 0}
taxidata['label'] = taxidata['label'].replace(mat_t)

# calculate distances
# testdata["distance"] = np.sqrt(np.sum(np.square([testdata["pick_lat"]-testdata["drop_lat"], testdata["pick_lon"]-testdata["drop_lon"]]), axis=0))*10000

taxidata.describe()

Feature Importance Analysis

In [ ]:
# define custom class to fix bug in xgboost 1.0.2
class MyXGBClassifier(XGBClassifier):
    @property
    def coef_(self):
        return None

#Drop objective coumns
selected_features = [feat for feat in taxidata.columns if feat not in ['pickup_time', 'drop_time', 'pick_lat', 'pick_lon', 'drop_lat', 'drop_lon']]
taxidata = taxidata[selected_features]
X = taxidata.iloc[:,:-1]
Y = taxidata.iloc[:,-1:]

X['waiting_ratio'] = X['meter_waiting']/(X['meter_waiting_fare']+1)
selected_features.append('waiting_ratio')

# split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.33, random_state=RANDOM_SEED)
X_train.describe()
# fit model on all training data
model = MyXGBClassifier()
model.fit(X_train, y_train)
plot_importance(model)
pyplot.show()
# make predictions for test data and evaluate
predictions = model.predict(X_test)
accuracy = accuracy_score(y_test, predictions)
print("Accuracy: %.2f%%" % (accuracy * 100.0))
# Fit model using each importance as a threshold
thresholds = sort(model.feature_importances_)
for thresh in thresholds:
    # select features using threshold
    selection = SelectFromModel(model, threshold=thresh, prefit=True)
    select_X_train = selection.transform(X_train)
    # train model
    selection_model = XGBClassifier()
    selection_model.fit(select_X_train, y_train)
    # eval model
    select_X_test = selection.transform(X_test)
    predictions = selection_model.predict(select_X_test)
    accuracy = accuracy_score(y_test, predictions)
    print("Thresh=%.3f, n=%d, Accuracy: %.2f%%" % (thresh, select_X_train.shape[1], accuracy*100.0))

Smote Analysis

In [ ]:
#Drop objective coumns
selected_features = [feat for feat in taxidata.columns if feat not in ['pickup_time', 'drop_time', 'pick_lat', 'pick_lon', 'drop_lat', 'drop_lon']]
taxidata = taxidata[selected_features]

X = taxidata.iloc[:,:-1]
Y = taxidata.iloc[:,[-1]].values[:,0]

X_train, X_eval, y_train, y_eval = train_test_split(
    X,
    Y,
    test_size=0.33,
    shuffle=True,
    stratify=labels_df,
    random_state=RANDOM_SEED
)

numeric_preprocessing_steps = Pipeline([
    ('standard_scaler', StandardScaler()),
    ('simple_imputer', SimpleImputer(strategy='median'))
])
preprocessor = ColumnTransformer(
    transformers = [
        ("numeric", numeric_preprocessing_steps, categorical),
    ],
    remainder = "drop"
)

model = estimators = estimator=XGBClassifier(
                    objective='binary:logistic',
                    booster='gbtree',   #default from gbtree, gblinear or dart
                    learning_rate =0.05,   
                    n_estimators=1500,
                    max_depth=4,    #default 6, increase will overfit the model
                    min_child_weight=6,
                    gamma=1,
                    subsample=0.8,
                    colsample_bytree=0.8,
                    reg_alpha=0.005,    #default 1
                    reg_lambda=0,   #default 0
                    scale_pos_weight=1,
)
full_pipeline = Pipeline([
    ("preprocessor", preprocessor),
    ("estimators", estimators),
])
# values to evaluate
k_values = [1,2,3,4,5,6,7]
for k in k_values:
    over = SMOTE(sampling_strategy=0.1, k_neighbors=k)
    under = RandomUnderSampler(sampling_strategy=0.5)
    steps = [('over', over), 
#              ('under', under),
             ('model', model)]
    pipeline = Pipeline(steps=steps)
    # evaluate pipeline
    cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=3, random_state=1)
    print(cv.split(X, y))
    scores = cross_val_score(pipeline, X, y, scoring='f1_macro', cv=cv, n_jobs=-1)
    print(scores)
    score = mean(scores)
    print('> k=%d, Mean f1_macro: %.3f' % (k, score))

Select features based on importance & Remove Noisy Data

In [ ]:
# taxidata_noise = taxidata[\
# #                           (taxidata.fare<taxidata.additional_fare+taxidata.meter_waiting_fare)\
# #                           |(taxidata.duration<taxidata.meter_waiting)\
#                           (taxidata.duration>21.5)\   #21.5 min value for wrong label
#                           |(taxidata.additional_fare>=5)\   #5 min value for wrong label
#                           |(taxidata.meter_waiting<11723)\   #max value for wrong label
# #                           |((taxidata.distance==0)&(taxidata.meter_waiting==0))|\
#                            |(taxidata.fare>60)]    #60 min fare for wrong label
# taxidata = taxidata[(~taxidata.index.isin(taxidata_noise.index))]

# for col in taxidata.:
#     taxidata = taxidata[taxidata[col].between(taxidata1[col].quantile(0.01), taxidata1[col].quantile(0.99))]

# taxidata = taxidata[(taxidata.fare>taxidata.additional_fare+taxidata.meter_waiting_fare)| (taxidata.label==False)]
# taxidata = taxidata[(taxidata.fare!=0)| (taxidata.label==False)]
# taxidata = taxidata[taxidata.additional_fare==10.5]
# taxidata = taxidata[taxidata.meter_waiting_fare<105]
# taxidata = taxidata[taxidata.meter_waiting_fare<140]
# taxidata = taxidata[taxidata.meter_waiting_till_pickup<510]
# taxidata = taxidata[taxidata.meter_waiting<2400]
# taxidata = taxidata[taxidata.duration<6500]
# taxidata = taxidata[taxidata.duration.between(taxidata.duration.quantile(.01),taxidata.duration.quantile(.99))]
# taxidata = taxidata[(taxidata.fare!=0)| (taxidata.label==False)]
# taxidata = taxidata[taxidata.fare<2000]
# taxidata = taxidata[(taxidata.meter_waiting_till_pickup<1510)]
# taxidata = taxidata[(taxidata.meter_waiting<14000)]
# taxidata = taxidata[(taxidata.duration<16500)]
# taxidata = taxidata[(taxidata.fare<10000)]


# taxidata = taxidata[(taxidata.meter_waiting_fare<1005)]
# taxidata = taxidata[(taxidata.meter_waiting_till_pickup<1510)]
# taxidata = taxidata[(taxidata.meter_waiting<14000)]
# taxidata = taxidata[(taxidata.duration<16500)]
# taxidata = taxidata[(taxidata.fare<10000)]
# taxidata.fare = taxidata.fare - taxidata.additional_fare
# taxidata = taxidata.drop('additional_fare', axis=1)
# taxidata = taxidata[taxidata.fare>taxidata.additional_fare]


# minmax = {}
# for index in taxidata.columns:
#     if index != 'label':
#         min0 = taxidata[taxidata.label==0][index].min()
#         max0 = taxidata[taxidata.label==0][index].max()
#         min1 = taxidata[taxidata.label==1][index].min()
#         max1 = taxidata[taxidata.label==1][index].max()
#         minmax[index] = (min0 if min0>min1 else min1, max0 if max0<max1 else max1)
# #         minmax[index] = (min1, max1)
#         print(minmax[index])
#         taxidata = taxidata[(minmax[index][0]<taxidata[index])&(taxidata[index]<minmax[index][1])]
# taxidata.describe()



In [ ]:
#Drop objective coumns

selected_features = [feat for feat in taxidata.columns if feat not in ['pickup_time', 'drop_time', 'pick_lat', 'pick_lon', 'drop_lat', 'drop_lon']]
taxidata = taxidata[selected_features]

taxidata['waiting_ratio'] = taxidata['meter_waiting']/(taxidata['meter_waiting_fare']+1)

#Drop least influence features
selected_features = [feat for feat in taxidata.columns if feat not in ['meter_waiting_fare', 'additional_fare',]] # 'duration_t']]
taxidata = taxidata[selected_features]

Hyper Param Tuning Functions

In [ ]:
from hyper_param_tuning import *
X = taxidata.iloc[:,:-1]
Y = taxidata.iloc[:,[-1]]
# split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.33, random_state=7)

data0 = { "x_train" : X_train,
         "y_train" : y_train,
         "x_test"  : X_test,
         "y_test"  : y_test}

#%%
param_grid_dic = OrderedDict([
        ("learning_rate", [0.05, 0.10, 0.15, 0.20, 0.25, 0.30 ] ),
        ("max_depth"    , [  3 , 4 , 5, 6,  8, 10, 12, 15 ] ),
        ("min_child_weight", [ 1, 3, 5, 7 ] ),
        ("gamma", [0.0, 0.1, 0.2, 0.3, 0.4 ]),
        ("colsample_bytree", [  0.3, 0.4, 0.5, 0.7 ] ),
        ])

#%%
train_fraction = 0.05
test_fraction = 0.16
data = subsample( data0, train_fraction, test_fraction )
#%%
seed=1359
log_level=0
#%%
memoization_path = "xgboost_memo%g" % train_fraction
print( "memoization_path= " + memoization_path)
if not os.path.exists( memoization_path ) :
    os.mkdir( memoization_path )
#%%
fun = Memoizer( lambda param_dic : train_xgb( data, param_dic),
                memoization_path )
#%%
grid_search( param_grid_dic, fun , seed)
#%%

Preprocess Data

In [ ]:
X = taxidata.iloc[:,:-2]
X['waiting_ratio'] = taxidata['waiting_ratio']
Y = taxidata.iloc[:,-2:-1]

# split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.33, random_state=7)

np.testing.assert_array_equal(X.index.values, Y.index.values)

null_columns=X.columns[X.isnull().any()]
print("columns contains null data", X[null_columns].isnull().sum())

Fit Model and Evaluate

In [ ]:
# chain preprocessing into a Pipeline object
# each step is a tuple of (name you chose, sklearn transformer)
numeric_preprocessing_steps = Pipeline([
    ('standard_scaler', StandardScaler()),
    ('simple_imputer', SimpleImputer(strategy='median'))
])

if 'label' in selected_features: selected_features.remove('label')
preprocessor = ColumnTransformer(
    transformers = [
        ("numeric", numeric_preprocessing_steps, selected_features),
    ],
    remainder = "drop"
)

taxidata.describe()

full_pipeline = Pipeline([
    ("preprocessor", preprocessor),
])

X_train = full_pipeline.fit(X_train)

# over = SMOTE(sampling_strategy=0.5, k_neighbors=7)
# print(Counter(Y.label))
# X, Y = over.fit_resample(X, Y)
# print(Counter(Y.label))
X_train, X_eval, y_train, y_eval = train_test_split(
    X,
    Y,
    test_size=0.33,
    shuffle=True,
    stratify=Y,
#     random_state=RANDOM_SEED
)

X_train.shape,y_train.shape

# over = SMOTE(sampling_strategy=0.5, k_neighbors=7)
# print(Counter(y_train.label))
# X_train, y_train = over.fit_resample(X_train, y_train)
# print(Counter(y_train.label))

# under = RandomUnderSampler(sampling_strategy=0.1)
# print(Counter(y_train.label))
# X_train, y_train = under.fit_resample(X_train, y_train)
# print(Counter(y_train.label))

def f1_eval(y_pred, dtrain):
    y_true = dtrain.get_label()
    err = 1-f1_score(y_true, np.round(y_pred))
    return 'f1_err', err

# estimate scale_pos_weight value
counter = Counter(Y.label)
est_scale_pos_weight = counter[0] / counter[1]

eval_set = [(X_train, y_train), (X_eval, y_eval)]

# model = MultiOutputClassifier(estimator = 
model = XGBClassifier(
        booster='gbtree',   #default from gbtree and gblinear or dart
        learning_rate =0.05,   
        n_estimators=2500,
        max_depth=3,    #default 6, increase will overfit the model [  3 , 4 , 5, 6,  8, 10, 12, 15 ]
        min_child_weight=6,    # [ 1, 3, 5, 7 ] 
        gamma=0.2,
        subsample=1,
        colsample_bytree=1,    # [  0.3, 0.4, 0.5, 0.7 ]
        reg_alpha=0.005,
        reg_lambda=0.09,   #default 0 for "majority class set to 0"
        objective='binary:logistic',
        scale_pos_weight= 0.35,
        sampling_method = 'gradient_based',
#         sample_type='weighted',    #for dart only
#         rate_drop = 0.1,    #for dart only
#         seed=27
    verbosity = 0,
    )
# )

model.fit(X_train, y_train, early_stopping_rounds=10, eval_metric=["error", "logloss"], eval_set=eval_set, verbose=True)

predictions = model.predict(X_eval)

# evaluate predictions
accuracy = accuracy_score(y_eval, predictions)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

print(model.score(X_eval,y_eval))
print(confusion_matrix(y_eval,predictions))
print(classification_report(y_eval,predictions))

# retrieve performance metrics
results = model.evals_result()
epochs = len(results['validation_0']['error'])
x_axis = range(0, epochs)
# plot log loss
fig, ax = pyplot.subplots()
ax.plot(x_axis, results['validation_0']['logloss'], label='Train')
ax.plot(x_axis, results['validation_1']['logloss'], label='Test')
ax.legend()
pyplot.ylabel('Log Loss')
pyplot.title('XGBoost Log Loss')
pyplot.show()
# plot classification error
fig, ax = pyplot.subplots()
ax.plot(x_axis, results['validation_0']['error'], label='Train')
ax.plot(x_axis, results['validation_1']['error'], label='Test')
ax.legend()
pyplot.ylabel('Classification Error')
pyplot.title('XGBoost Classification Error')
pyplot.show()


Predict Unlabeled Data

In [ ]:
null_columns=testdata.columns[testdata.isnull().any()]
testdata[null_columns].isnull().sum()

testdatadup = testdata[:]

testdatadup['waiting_ratio'] = testdatadup['meter_waiting']/(testdatadup['meter_waiting_fare']+1)

testdata0 = testdatadup[(testdatadup.fare<testdatadup.additional_fare+testdatadup.meter_waiting_fare)|(testdatadup.duration<testdatadup.meter_waiting)|(testdatadup.duration==0)|((testdatadup.distance==0)&(testdatadup.meter_waiting==0))|(testdatadup.fare==0)]
testdata1 = testdatadup[(~testdatadup.index.isin(testdata0.index))]
testdata0.shape

if 'label' in selected_features: selected_features.remove('label')
testdata1 = testdata1[selected_features]
testdata0 = testdata0[selected_features]
testdata0['prediction'] = 0
test_probas = model.predict(testdata1)

# Save predictions to submission data frame
testdata1["prediction"] = test_probas
# print(testdata0.describe())
# print(testdata1.describe())
predicted_df=testdata1.append(testdata0)
# print(predicted_df.describe())
submission_df = pd.read_csv('../input/sample/sample_submission.csv',index_col='tripid')
predicted_df = predicted_df.reindex(submission_df.index)


# Make sure we have the rows in the same order
np.testing.assert_array_equal(predicted_df.index.values, submission_df.index.values)

print(sum(1 for val in predicted_df.prediction if val==1))
print(sum(1 for val in predicted_df.prediction if val==0))
predicted_df = predicted_df[['prediction']]
predicted_df.to_csv('160256U_predicted_output.csv', index=True)